In [3]:
import numpy as np
import modules
from scipy.optimize import minimize

def sensitivity_sweep(v1_name, v1_ratios, v2_name):
    # v1: variable being swept, held constant for each optimization
    # v2: variable being optimized

    v1_list = modules.variable_lookup(v1_name)
    v2_list = modules.variable_lookup(v2_name);
    v1_nom = modules.default_values(v1_name);
    num_v1 = len(v1_list);
    num_v2 = len(v2_list);
    num_ratios = len(v1_ratios);
    
    v2_star = np.zeros(shape=(num_v1,num_ratios,num_v2))
    dv2_star_dv1 = np.zeros(shape=(num_v1,num_v2))
        
    for i in range(num_v1):
        for j in range(num_ratios):
            v1 = v1_nom
            v1[v1_list[i]] = v1_nom[v1_list[i]] * v1_ratios[j]
            res_opt = run_optimization(v2_name,v1_name,v1)
            #print(res_opt)
            v2_star[i,j,:] = res_opt['x']
            #print('>>>>>',v2_star[i,j,:])

        idx_nom = ratios.index(1)
        dv2_star = np.divide((v2_star[i,-1,:] - v2_star[i,0,:]), v2_star[i,idx_nom,:])
        dv2_star_dv1[i,:] = dv2_star / (v1_ratios[-1] - v1_ratios[0])    

    return dv2_star_dv1


def run_optimization(x_name,p_name,p_vals):
    # optimizes the design variables x_name, with parameters p_name set to 
    # non-default values p_vals, and other parameters set to default values.
    
    # create optimization variable
    x_list = modules.variable_lookup(x_name)
    x_label = ''
    for i in range(len(x_name)):
        x_label += x_name[i] + '%'
    x = [x_label,x_list]

    x_struct={}
    # create struct to hold optimization variable
    for i in range(len(x_list)):
        x_struct[x_list[i]] = 0 #x[x_list[i]]
    
    # fill default parameters
    all_vars = ['x_wec','x_wec_type','x_pen','p_pen','x_env','p_fish','p_wec']
    str_ = x_name + p_name
    
    default_vars = []
    for i in range(len(all_vars)):
        if all_vars[i] not in str_:
            default_vars.append(all_vars[i])
    #is_default = ~contains(all_vars,[x_name p_name]);
    #default_vars = all_vars[is_default]
    p = modules.default_values(default_vars)

    # fill non-default parameters
    p_label = ''
    if p_name!=[]:
        for i in range(len(p_name)):
            p_label += p_name[i] + '%'
        p_list = modules.variable_lookup(p_name);
        for i in range(len(p_list)):
            p[p_list[i]] = p_vals[p_list[i]]

    desc = 'Finding optimal '+ x_label + ' while holding '+ p_label + ' constant.'
    print(desc)
    
    # design variables
    x_list_default_values = modules.default_values(x_name)
    x0 = []
    for i in range(len(x_list)):
        x0.append(x_list_default_values[x_list[i]])
        
    # set up optimization problem
    J = modules.obj(x0, x_name, p)
    
    #optimization
    arguments = (x_name, p)
    cons = ({'type': 'ineq', 'fun': modules.ineq_constraint, 'args': arguments},
            {'type': 'eq', 'fun': modules.eq_constraint, 'args': arguments})
    bnds = tuple((0,1) for x in x0)
    res = minimize(modules.obj, x0, args=arguments, method='SLSQP', constraints=cons)

    return res



ratios = [.8, 1, 1.2]
# sensitivity of optimal wec and pen design to environment/location
dx_wec_pen_star_dx_env = sensitivity_sweep(['x_env'], ratios, ['x_wec','x_pen'])
print('\n\ndx_wec_pen_star_dx_env=\n', dx_wec_pen_star_dx_env)
print('*************************************************\n\n')

# sensitivity of optimal environment/location to wec and pen design
dx_env_star_dx_wec_pen = sensitivity_sweep(['x_wec','x_pen'], ratios, ['x_env'])
print('\n\ndx_env_star_dx_wec_pen=\n', dx_env_star_dx_wec_pen)
print('*************************************************\n\n')

Finding optimal x_wec%x_pen% while holding x_env% constant.
Finding optimal x_wec%x_pen% while holding x_env% constant.
Finding optimal x_wec%x_pen% while holding x_env% constant.
Finding optimal x_wec%x_pen% while holding x_env% constant.
Finding optimal x_wec%x_pen% while holding x_env% constant.
Finding optimal x_wec%x_pen% while holding x_env% constant.
Finding optimal x_wec%x_pen% while holding x_env% constant.
Finding optimal x_wec%x_pen% while holding x_env% constant.
Finding optimal x_wec%x_pen% while holding x_env% constant.
Finding optimal x_wec%x_pen% while holding x_env% constant.
Finding optimal x_wec%x_pen% while holding x_env% constant.
Finding optimal x_wec%x_pen% while holding x_env% constant.
Finding optimal x_wec%x_pen% while holding x_env% constant.
Finding optimal x_wec%x_pen% while holding x_env% constant.
Finding optimal x_wec%x_pen% while holding x_env% constant.
Finding optimal x_wec%x_pen% while holding x_env% constant.
Finding optimal x_wec%x_pen% while holdi